In [2]:
import glob
import multiprocessing
from os import path

import numpy as np
import pandas as pd
import pyproj
import tensorflow as tf
from keras import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.layers import Dense, Dropout, Normalization
from keras.losses import Huber, MeanSquaredError
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.regularizers import L2
from keras_tuner import Objective, HyperParameters
from keras_tuner.tuners import BayesianOptimization, Hyperband, RandomSearch
from orbslam_results import ORBSLAMTrajectoryProcessor
from plotly import graph_objects as go

from utils import denormalize, reshape_data, umeyama_alignment


TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'


processor = ORBSLAMTrajectoryProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))


In [12]:
def build_model(hp: HyperParameters):
    model = Sequential()

    # Input layer
    model.add(Dense(units=hp.Int('input_layer_units', min_value=16, max_value=256, step=16),
                    activation=hp.Choice('input_layer_0_activation', values=['relu', 'selu', 'elu', 'tanh']),
                    input_shape=(3,)))

    num_hidden_layers = hp.Int('num_hidden_layers', 0, 5)
    # Choose the number of hidden layers
    for i in range(num_hidden_layers):
        # Tune the number of nodes and activation function for each layer
        model.add(Dense(units=hp.Int(f'hidden_layer_{i}_units', min_value=16, max_value=256, step=16),
                        activation=hp.Choice(f'hidden_layer_{i}_activation', values=['relu', 'selu', 'elu', 'tanh'])))
        # Tune whether to use dropout.
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=hp.Choice(f'hidden_layer_{i}_dropout_rate', values=[0.5, 0.6, 0.7, 0.8])))

    # Output layer
    model.add(Dense(3))

    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    loss = hp.Choice('loss', values=['mean_squared_error', 'huber', 'mean_absolute_error', 'log_cosh'])

    # Compile the model
    model.compile(optimizer=opt, loss=loss)

    return model


def predict_trajectory(source_points):
    print(f"Predicting target trajectory with {source_points.shape}")

    source = reshape_data(source_points, self.n_inputs)
    print(f"Resized source: {source.shape}")

    normalized_source_points = self._source_normalizer(source)

    X = normalized_source_points

    # Predict the target trajectory
    normalized_predicted_target_points = self._model.predict(X)
    denormalized_predicted_target_points = denormalize(self._target_normalizer, normalized_predicted_target_points)

    return denormalized_predicted_target_points.reshape(-1, 3)


def fit():
    self._fit_trajectory(self.orbslam.mapping.trajectory, self.orbslam.mapping.trajectory_gt_utm)
    self.orbslam.mapping.trajectory_utm = self.predict_trajectory(self.orbslam.mapping.trajectory)
    for localization in self.orbslam.localization.values():
        localization.trajectory_utm = self.predict_trajectory(localization.trajectory)


Training model with (535, 3)
Target points: (535, 3)
Resized source:  (535, 3)
Resized target:  (535, 3)


2023-04-01 15:47:04.164903: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


INFO:tensorflow:Reloading Tuner from keras_logs/vabadusepst/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/400
67/67 [==============================] - 0s 657us/step - loss: 0.0776
Epoch 2/400
67/67 [==============================] - 0s 718us/step - loss: 0.0357
Epoch 3/400
67/67 [==============================] - 0s 661us/step - loss: 0.0318
Epoch 4/400
67/67 [==============================] - 0s 690us/step - loss: 0.0284
Epoch 5/400
67/67 [==============================] - 0s 704us/step - loss: 0.0253
Epoch 6/400
67/67 [==============================] - 0s 692us/step - loss: 0.0281
Epoch 7/400
67/67 [==============================] - 0s 677us/step - loss: 0.0413
Epoch 8/400
67/67 [==============================] - 0s 665us/step - loss: 0.0340
Predicting target trajectory with (535, 3)
Resized source: (535, 3)
17/17 [==============================] - 0s 440us/step
Predicting target trajectory with (2938, 3)
Resized source: (2938, 3)
92/92 [==============================] - 

In [18]:

def split_data(data):
    validation_slice = slice(0, None, 4)
    validation = data[validation_slice]
    training = np.delete(data, validation_slice, axis=0)
    
    return training, validation


training, validation = split_data(processor.orbslam.mapping.trajectory_gt_utm)
print(processor.orbslam.mapping.trajectory_gt_utm.shape)
print(training.shape)
print(validation.shape)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=processor.orbslam.mapping.trajectory_gt_utm[:, 0],
        y=processor.orbslam.mapping.trajectory_gt_utm[:, 1],
        mode='markers',
        name='trajectory_gt_utm'
    )
)
fig.add_trace(
    go.Scatter(
        x=training[:, 0],
        y=training[:, 1],
        mode='markers',
        name='training'
    )
)
fig.add_trace(
    go.Scatter(
        x=validation[:, 0],
        y=validation[:, 1],
        mode='markers',
        name='validation'
    )
)

fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1), height=1200)
fig.show()


(535, 3)
(356, 3)
(179, 3)


(535, 3)
(134, 3)
(401, 3)
[[4.83815228e+05 6.47155051e+06 3.57959651e+01]
 [4.83815163e+05 6.47155013e+06 3.57912773e+01]
 [4.83814985e+05 6.47154909e+06 3.57664787e+01]
 ...
 [4.83908245e+05 6.47114538e+06 3.57411396e+01]
 [4.83908459e+05 6.47114487e+06 3.57554406e+01]
 [4.83908474e+05 6.47114487e+06 3.57567792e+01]]
